In [5]:
using OhMyJulia
using DataFrames
using PyCall
using NaturalSort
@pyimport gurobipy as grb

include("path.jl")
include("weight.jl")
ENV["PATH"] *= ":/usr/local/gurobi/bin/:/home/zhangshiwei/.opam/4.06.0/bin"

data_list = ["abilene", "Cernet", "Cesnet201006", "Chinanet", "Esnet", "Garr200112", "Grnet", "Ibm", "jelly16", "Nordu1997", "Tinet"]
algo_list = ["Ecmp", "Edskp", "Ksp", "Mcf", "Raeck", "Vlb"]

6-element Array{String,1}:
 "Ecmp" 
 "Edskp"
 "Ksp"  
 "Mcf"  
 "Raeck"
 "Vlb"  

In [68]:
function gen_gravity(data)
    hosts = readlines("data/$data.hosts")
    open("data/$data.demands", "w") do fout
        for k in 1:6
            α = 2^(22 + k)
            weigths = map(x->rand(), hosts)
            list = [ α * weigths[i] * weigths[j]
                     for i in 1:length(hosts)
                     for j in 1:length(hosts) ]
            println(fout, join(list, ' '))
        end
    end
end
for i in data_list
    gen_gravity(i)
end

In [2]:
function read_topo(file)
    nodes, edges = Set{String}(), String[]
    for line in eachline(file) @when (m = match(r"(s\d+) -> (s\d+)", line)) != nothing
        push!(nodes, m[1], m[2])
        push!(edges, "$(m[1]) $(m[2])")
    end
    nodes, edges
end

function read_demand(hosts, demands)
    results = []
    hosts = readlines(hosts)
    n = length(hosts)
    for line in eachline(demands)
        line = map(x->parse(f64, x), split(line, ' '))
        demand = Dict{String, Float64}()
        for i in 1:n, j in 1:n @when i != j
            demand["$(hosts[i]) $(hosts[j])"] = line[(i-1) * n + j] / 2^20 # to Megabyte
        end
        push!(results, demand)
    end
    results
end

function read_budget(hosts, budgets)
    Dict(zip(map(x->replace(x, 'h', 's'), readlines(hosts)),
             map(x->parse(Int, x),        readlines(budgets))))
end

function write_scheme(io, scheme)
    for (pair, paths) in scheme
        println(io, pair, ':')
        for (path, w) in paths
            p = map(x->split(x, ' '), path)
            println(io, join(map(car, p) ++ cadr(p[end]), " -> "), " @ ", w)
        end
        println(io)
    end
end

write_scheme (generic function with 1 method)

In [ ]:
for data in ("jelly16",) #data_list
    io, process = open(`yates -budget 16 $data`)
    path_sets = parse_schemes(io)
    nodes, edges = read_topo("data/$data.dot")
    demands = read_demand("data/$data.hosts", "data/$data.demands")
    budgets = read_budget("data/$data.hosts", "data/$data.budgets")
    
    for (i, demand) in enumerate(demands), algo in ("Vlb",) # algo_list
        for (name, select) in (("program", select_program),
                               ("greedy", select_greedy),
                               ("hardnop", select_hard_nop))
            open("results/$data-$i-$algo-$name.result", "w") do fout
                raw_scheme = select(nodes, edges, path_sets[algo], budgets)
                m1_scheme, Z = minimize_maximum_link_utilization(nodes, edges, demand, raw_scheme)
                m2_scheme, Zs = minimize_maximum_link_utilization_then_maximize_throuput(nodes, edges, demand, raw_scheme)
                
                println(fout, "number of nodes (switch): ", length(nodes))
                println(fout, "number of edges (core): ", length(edges))
                println(fout, "number of demand pairs: ", length(demand))
                println(fout, "average budgets: ", mean(values(budgets)))
                println(fout)
                
                println(fout, "Z: ", Z)
                println(fout, "total throuput before the second step:", sum(values(demand)) / max(Z, 1))
                println(fout, "total throuput after the second step:", sum(d / Zs[pair] for (pair, d) in demand))
                println(fout)
                
                println(fout, "number of path through each node:")
                node_dict = Dict(n => 0 for n in nodes)
                for (pair, paths) in m2_scheme, (path, weight) in paths, edge in path
                    n = split(edge, ' ') |> car
                    if car(n) == 's'
                        node_dict[n] += 1
                    end
                end
                for (node, number) in sort(collect(node_dict), lt=natural, by=car)
                    println(fout, "  ", node, ": ", number)
                end
                println(fout, "  average: ", sum(values(node_dict)) / length(node_dict))
                println(fout)
                
                println(fout, "number of path, total flow, and link utiliaztion of each edge:")
                edge_dict = Dict(e => [] for e in edges)
                for (pair, paths) in m2_scheme, (path, weight) in paths, edge in path @when 'h' ∉ edge
                    push!(edge_dict[edge], demand[pair] * weight)
                end
                for (edge, flows) in sort(collect(edge_dict), lt=natural, by=car)
                    println(fout, "  ", edge, ": ", length(flows), ", ", sum(flows), ", ", sum(flows) / 100000, '%')
                end
                println(fout, "  average: ", sum(length(v) for v in values(edge_dict)) / length(edge_dict), 
                                       ", ", sum(sum(v) for v in values(edge_dict)) / length(edge_dict),
                                       ", ", sum(sum(v) for v in values(edge_dict)) / length(edge_dict) / 100000, '%')
                println(fout)
                
                println(fout, "number of path and demand ratio of each st pair:")
                pair_dict = Dict(pair => length(paths) for (pair, paths) in m2_scheme)
                for (pair, nflows) in sort(collect(pair_dict), lt=natural, by=car)
                    println(fout, "  ", pair, ": ", nflows, ", ", 1 / Zs[pair])
                end
                println(fout, "  average: ", sum(values(pair_dict)) / length(pair_dict),
                                       ", ", sum(d / Zs[pair] for (pair, d) in demand) / sum(values(demand)))
                println(fout)
                
                println(fout, "path set:")
                write_scheme(fout, path_sets[algo])
                println(fout)
                
                println(fout, "paths selected:")
                write_scheme(fout, raw_scheme)
                println(fout)
                
                println(fout, "paths after the first step (minimize maximum link utilization):")
                write_scheme(fout, m1_scheme)
                println(fout)
                
                println(fout, "paths after the second step (maximize total throuput):")
                write_scheme(fout, m2_scheme)
            end
        end
    end
end

In [3]:
data = "jelly16"
io, process = open(`yates -budget 16 $data`)
    path_sets = parse_schemes(io)

nodes, edges = read_topo("data/$data.dot")
    demands = read_demand("data/$data.hosts", "data/$data.demands")
    budgets = read_budget("data/$data.hosts", "data/$data.budgets")


Dict{String,Int64} with 16 entries:
  "s8"  => 240
  "s5"  => 240
  "s7"  => 240
  "s14" => 240
  "s9"  => 240
  "s10" => 240
  "s6"  => 240
  "s11" => 240
  "s15" => 240
  "s3"  => 240
  "s1"  => 240
  "s13" => 240
  "s12" => 240
  "s2"  => 240
  "s4"  => 240
  "s0"  => 240

In [6]:
x = select_program(nodes, edges, path_sets["Vlb"], budgets)
cadr(minimize_maximum_link_utilization_then_maximize_throuput(nodes, edges, demands[5], x))

h1 h14	59.13663532346369	40.831621442722465
h14 h13	57.62842478006224	39.79025881484809
h3 h5	29.38501230497247	20.289245270102757
h9 h12	7.955699255694169	7.955699255694169
h8 h10	34.77063816150299	32.14763469930274
h2 h13	76.47867831729327	52.805649567493234
h7 h4	0.43418879217233797	0.43418879217233797
h8 h5	22.37892868353335	22.37892868353335
h5 h9	26.91164943250866	18.581481276657286
h12 h13	15.63053769332805	10.792298116882346
h12 h0	3.9288606470174856	3.9288606470174856
h0 h10	20.649139283681485	14.257453670428882
h15 h8	37.89226474197427	26.16318295907221
h9 h0	9.301215660692472	9.301215660692472
h5 h11	2.94205223923953	2.0313763649261736
h5 h15	65.10891492823754	44.955255779345975
h5 h2	55.6203456661255	38.40375574853701
h1 h7	1.8277393899924637	1.261985272917851
h0 h1	18.752325686559335	17.37945763377394
h10 h12	17.66202913928345	17.66202913928345
h8 h4	7.5011902799664485	7.5011902799664485
h4 h7	0.43418879217233797	0.43418879217233797
h7 h1	1.8277393899924637	1.2619852729178

Dict{Any,Any} with 240 entries:
  "h1 h14"  => 1.4483
  "h14 h13" => 1.4483
  "h3 h5"   => 1.4483
  "h9 h12"  => 1.0
  "h8 h10"  => 1.08159
  "h2 h13"  => 1.4483
  "h7 h4"   => 1.0
  "h8 h5"   => 1.0
  "h5 h9"   => 1.4483
  "h12 h13" => 1.4483
  "h12 h0"  => 1.0
  "h0 h10"  => 1.4483
  "h15 h8"  => 1.4483
  "h9 h0"   => 1.0
  "h5 h11"  => 1.4483
  "h5 h15"  => 1.4483
  "h5 h2"   => 1.4483
  "h1 h7"   => 1.4483
  "h0 h1"   => 1.07899
  "h10 h12" => 1.0
  "h8 h4"   => 1.0
  "h4 h7"   => 1.0
  "h7 h1"   => 1.4483
  "h13 h10" => 1.4483
  "h2 h6"   => 1.0
  ⋮         => ⋮